<a href="https://colab.research.google.com/github/juniperlovato/privacypolicypaper/blob/main/negation_filtered_policytext_PII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import numpy as np
import array as ar

#--
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Summary

1.   We are going to want to feed in each year of the privacy policy text
2.   then we want to filter out clauses that only have negation words (save the csv here)
3.   then filter the data frame so it only has PII lexicon terms (save the csv here too)



In [ ]:
#we will want to feed in each year of priviacy policies
data_base_path = '/data'
cleaned_data_path = 'by-year'
data_path = os.path.join(data_base_path, cleaned_data_path)
#--
df = pd.read_csv(os.path.join(data_path, 'PrivacyPolicies-2019.csv'))

#negation

In [ ]:

df['list_of_clauses'] = df['policy_text'].astype('object')
df['nb_of_clauses'] = df['length']
df['list_of_clauses'].astype(str).str.lower()
df['nb_of_clauses'].astype(str).str.lower()

0         14137
1         14366
2         14363
3         14394
4         14396
          ...  
333981    11953
333982     5582
333983    16906
333984     4168
333985    18984
Name: nb_of_clauses, Length: 333986, dtype: object

In [ ]:
df_clauses = df[['nb_of_clauses', 'list_of_clauses']].copy()

In [ ]:
#we loop all rows and find all punctuation then split them each into a new column
#every entry in the new column now becomes a list of clauses

for index, row in df_clauses.iterrows():
    policy_texts_split = re.split(r"[!\"#\$%&\'\(\)\*\+,-\./:;<=>\?@\[\\\]\^_`{\|}~]", df_clauses.at[index,'list_of_clauses'])
    policy_texts_split = list(filter(None, policy_texts_split))
    df_clauses.at[index,'list_of_clauses'] = policy_texts_split
    df_clauses.at[index,'nb_of_clauses'] = len(policy_texts_split)


In [ ]:
# df_clauses

,nb_of_clauses,list_of_clauses
0,702,"[Effective Date, June 11, 2018\n\nAt DocuSi..."
1,702,"[Effective Date, June 11, 2018\n\nAt DocuSi..."
2,373,"[PDF, XChange Co Ltd , Tracker Software Produ..."
3,590,"[ INTRODUCTION\n\nWelcome to FCA US LLC , FCA ..."
4,1071,"[MH Sub I, LLC dba Internet Brands and its af..."
...,...,...
51786,52,"[Do we use cookies, \n\nWe do not use cookies,..."
51787,148,"[1 INTRODUCTION\n\n1, 1, PURPOSE OF POLICY\n\..."
51788,99,[This Privacy Policy governs the manner in whi...
51789,342,"[HUGEDOMAINS, COM PRIVACY POLICY\n\nHugeDomain..."


In [ ]:
#negation words from Juniper and Peter
negation_words = ["don't","never", "nothing", "nowhere", "noone", "none", "not",
                  "hasn't","hadn't","can't","couldn't","shouldn't","won't",
                  "wouldn't","don't","doesn't","didn't","isn't","aren't","ain't", "in*",  "un*", "dis*", "mal*"]


#“don't|never| nothing|nowhere|noone|none|not|hasn't|hadn't|can't|couldn't|shouldn't|won't|wouldn't|don't|doesn't|didn't|isn't|aren't|ain't|in*|un*|dis*|mal*”

In [ ]:
#iterate through all the rows
#look at list items that contain negation words
#if the list item contains a negation word then remove the list item (sentence)
#return a column with just the list of sentences that do not have negation words in them

def remove_negations(text):
    # Step 1: split the text into sentences
    sent = re.split(r'[.!]', text)
    #list to store sentences without negataions
    w_neg = []
    #step 2: check for each sentence in row
    for i in sent:
      neg = False
      #step 3: check for each negation word in the list of negation words
      for j in negation_words:
        if j in i and != PII: #if there is a negataion word in sentence
          neg = True #make it true
          break; #break the loop and go for another sentence
      if not neg: #but if it's still false meaning that no negation word was found in a sentence
        w_neg.append(i) #add that sentence to the list

    return w_neg #now join back the sentences

df['negation_Extracted'] = df['policy_text'].apply(remove_negations) #apply at each row



In [ ]:
# df['negation_Extracted']

0        [Effective Date:  June 11, 2018\n\nAt DocuSign...
1        [Effective Date:  June 11, 2018\n\nAt DocuSign...
2        [PDF-XChange Co Ltd & Tracker Software Product...
3        [## INTRODUCTION\n\nWelcome to FCA US LLC ("FC...
4        [MH Sub I, LLC dba Internet Brands and its aff...
                               ...                        
51786    [\n\nVisitor Stats\n\nWe are using two differe...
51787    [1 INTRODUCTION\n\n1, 1,  PURPOSE OF POLICY\n\...
51788    [This Privacy Policy governs the manner in whi...
51789    [HUGEDOMAINS, COM PRIVACY POLICY\n\nHugeDomain...
51790    [We, Cricket Traveling Ltd, are committed to p...
Name: negation_Extracted, Length: 51791, dtype: object

In [ ]:
#save it as a csv
#make sure to change the file name for the correct year

df.to_csv('ALLYEARS_negation_extracted.csv')

#filter PII

In [ ]:
#these are all of the terms in our PII lexicon put into a list
PII = ['Identifiers',
    'alias',
    'online identifier',
    'Internet Protocol (IP) address',
    'account name',
    'social security number',
    'passport number',
    'Customer records information',
    'IDENTIFICATION NUMBER',
    'signature',
    'electronic mail address',
    'address',
    'telephone number',
    'PROTECTED HEALTH INFORMATION',
    'state identification card number',
    'education',
    'employment history',
    'bank account number',
    'face',
    'financial information',
    'Records of personal property',
    'products purchased',
    'health condition',
    'consuming histories',
    'Services purchased',
    'eye color',
    'retina scans',
    'network activity',
    'Internet  activity',
    'search history',
    'Geolocation',
    'visual',
    'thermal',
    'olfactory',
    'Professional',
    'Medical Condition',
    'Characteristics',
    'Aggregated data',
    'Predispositions',
    'Behavior',
    'specific location',
    'facial recognition',
    'physiological',
    'behavioral',
    'Audio',
    'DNA',
    'iris',
    'retina',
    'hand',
    'palm',
    'vein patterns',
    'voice recordings',
    'minutiae template',
    'HEALTH STATUS',
    'keystroke patterns ',
    'keystroke rhythms',
    'gait rhythms',
    'sleep',
    'health',
    'exercise',
    'Cross-context behavioral advertising',
    'targeted advertising',
    'Dark pattern',
    'Personal information',
    'RACIAL',
    'real name',
    'Account number',
    'postal address',
    'financial account number',
    'Internet Protocol address',
    'Gender identity',
    'email address',
    'security question',
    'Color',
    'Religion',
    'Sex',
    'Sexual orientation',
    'Marital status',
    'National origin',
    'Ancestry',
    'Genetic information',
    'Retaliation for reporting patient abuse in tax-supported institutions',
    'Age',
    'religious dress',
    'pregnancy',
    'Gender',
    'childbirth',
    'breastfeeding',
    'mental characteristics',
    'physical characteristics',
    'HIV/AIDS',
    'cancer',
    'genetic characteristics',
    'Geolocation data',
    'record of cancer',
    'history of cancer',
    'gender expression',
    'abilities',
    'MENTAL CONDITION',
    'PREDICT',
    'biological',
    'purchasing tendencies',
    'Aggregate consumer information',
    'first name',
    'voice',
    'electronic network activity',
    'biological characteristic',
    'interaction with an advertisement',
    'Browsing history',
    'employment',
    'Race',
    'health records',
    'citizenship',
    'Military or veteran status',
    'medical identification number',
    'access code',
    'preferences',
    'protected classifications',
    'psychological trends',
    'Commercial information',
    'medical information',
    'attitudes',
    'intelligence',
    'Name',
    'driver\'s license',
    'Sensitive personal information',
    'Precise geolocation',
    'locate',
    'geographic area',
    'radius',
    'Sensitive data',
    'Profiling',
    'consumer\'s social security',
    'driver\'s license',
    'state Identification card',
    'account log-In',
    'credit card',
    'health insurance information',
    'password',
    'credentials allowing access to an account',
    'combination',
    'racial origin',
    'ethnic origin',
    'philosophical beliefs',
    'union membership',
    'text messages',
    'genetic data',
    'biometric data',
    'Personally identifiable information',
    'security code',
    'fingerprint',
    'device identifier',
    'IP address',
    'cookies',
    'beacons',
    'pixel tags',
    'customer number',
    'unique pseudonym',
    'telephone numbers',
    'persistent identifier',
    'probabilistic identifier',
    'family',
    'child',
    'identifier template',
    'DE-IDENTIFIED DATA',
    'HEALTH-CARE INFORMATION',
    'HEALTH-CARE PROVIDER',
    'MEDICINE',
    'PHARMACY',
    'CHIROPRACTIC',
    'NURSING',
    'PHYSICAL THERAPY',
    'PODIATRY',
    'DENTISTRY',
    'OPTOMETRY',
    'OCCUPATIONAL THERAPY',
    'HEALING ARTS',
    'IDENTIFIED',
    'IDENTIFIABLE INDIVIDUAL',
    'ONLINE IDENTIFIER',
    'PERSONAL DATA',
    'products obtained',
    'AUTOMATED PROCESSING',
    'Request for Pregnancy Disability Leave',
    'ANALYZE',
    'ECONOMIC SITUATION',
    'PERSONAL PREFERENCES',
    'religious beliefs',
    'RELIABILITY',
    'LOCATION',
    'MOVEMENTS',
    'PHYSICAL HEALTH CONDITION',
    'DIAGNOSIS',
    'credit card number',
    'CITIZENSHIP STATUS',
    'GENETIC',
    'last name',
    'mobile ad identifiers',
    'HEALTH-CARE',
    'PATIENT',
    'fingerprints',
    'products considered',
    'physical description',
    'voiceprint',
    'eye retinas',
    'Unique identifier',
    'consuming tendencies',
    'faceprint',
    'driver\'s license number',
    'Services considered',
    'global positioning system',
    'latitude',
    'longitude',
    'coordinates',
    'interests',
    'financial account',
    'user alias',
    'irises',
    'Request for leave for an employee\'s own serious health condition',
    'condition',
    'physical',
    'diagnosis',
    'insurance policy number',
    'immigration status',
    'known child',
    'sex life',
    'height',
    'AIDS',
    'medical diagnosis',
    'religious grooming practices',
    'Identifiable individual',
    'debit card',
    'ethnic',
    'origin',
    'medical treatment',
    'Inferences',
    'medical history',
    'mental health',
    'physical health',
    'mental',
    'Biometric information',
    'email content',
    'physical representation',
    'biological pattern',
    'mother\'s maiden name',
    'interaction with an internet website application',
    'deoxyribonucleic acid',
    'purchasing histories',
    'Disability',
    'targeting of advertising',
    'movements',
    'Hair color',
    'digital representation',
    'Initials',
    'SPECIFIC GEOLOCATION DATA',
    'driver authorization card number',
    'identification card number',
    'debit card number',
    'health insurance identification number',
    'user name',
    'Request for family care leave',
    'date of birth',
    'place of birth',
    'unique biometric',
    'human body',
    'HIV',
    'biometric',
    'language',
    'household',
    'driver\'s license number',
    'government-issued identification number',
    'driver license',
    'nondriver State identification card number',
    'individual taxpayer identification number',
    'military identification card number',
    'gait patterns',
    'unique personal identifier',
    'passwords',
    'personal identification number',
    'services obtained',
    'wellness program',
    'health promotion',
    'disease prevention',
    'health insurance policy number']


In [ ]:
#make sure the lexixon terms are lower case
for i in range(len(PII)):
    PII[i] = PII[i].lower()

In [ ]:
df['negation_Extracted'] = df['negation_Extracted'].astype(str)

In [ ]:
#filter out all of the PII terms and put them in a new column

extracted =[]
for index, row in df.iterrows():
    tempList =[]
    rowSplit = row['negation_Extracted'].split(" ")
    for val in rowSplit:
        if val in PII:
            tempList.append(val)
    if len(tempList)>0:
        extracted.append(' '.join(tempList))
    else:
        extracted.append(np.nan)

df['PII_Extracted'] = extracted

In [ ]:
#drop nans

df = df[df['PII_Extracted'].notna()]

In [ ]:
# df['PII_Extracted']

0        professional signature name geolocation identi...
1        professional signature name geolocation identi...
2        analyze location location language preferences...
3        combination locate address location locate add...
4        address physical characteristics health locati...
                               ...                        
51786                                              cookies
51787                            name name cookies analyze
51788                     cookies cookies cookies physical
51789    physical preferences name cookies cookies pref...
51790    analyze identified name address cookies langua...
Name: PII_Extracted, Length: 50000, dtype: object

In [ ]:
#make sure to change the file name for the correct year

df.PII_Extracted.to_csv('2019_pii_extracted.csv')